In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import ElasticNet
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

In [8]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [9]:
x_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [10]:
x_train = x_train.values
y_train = y_train.values
x_val = x_val.values
y_val = y_val.values
x_test = x_test.values
y_test = y_test.values

In [11]:
model = joblib.load("/home/jovyan/Edge-IIoT/dl/dl_new/dnn.joblib")

2024-02-18 10:58:43.346210: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://c1cfa78b6f204d0fbe8b9f2c6eab5c8e: INVALID_ARGUMENT: ram://c1cfa78b6f204d0fbe8b9f2c6eab5c8e is a directory.


In [12]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [24]:
x_test_trimmed = x_test[100:400]
y_test_trimmed = y_test[100:400]

In [14]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=15,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [25]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.001)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [1:09:56<00:00, 13.99s/it]


In [26]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 925us/step
Accuracy: 0.28
AUC: 0.5398025140205761

macro
Precision: 0.8802259887005649
Recall: 0.13830227743271223
F1 Score: 0.11239399869536856

weighted
Precision: 0.7579661016949153
Recall: 0.28
F1 Score: 0.19208414872798435

Mean FNR: 0.8616977225672877
Mean TNR: 0.9417711346833816
Mean FPR: 0.058228865316618444
Mean TPR: 0.13830227743271223


In [27]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [1:10:02<00:00, 14.01s/it]


In [28]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 798us/step
Accuracy: 0.28
AUC: 0.5398025140205761

macro
Precision: 0.8802259887005649
Recall: 0.13830227743271223
F1 Score: 0.11239399869536856

weighted
Precision: 0.7579661016949153
Recall: 0.28
F1 Score: 0.19208414872798435

Mean FNR: 0.8616977225672877
Mean TNR: 0.9417711346833816
Mean FPR: 0.058228865316618444
Mean TPR: 0.13830227743271223


In [29]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [1:09:59<00:00, 14.00s/it]


In [30]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 777us/step
Accuracy: 0.28
AUC: 0.5398025140205761

macro
Precision: 0.8802259887005649
Recall: 0.13830227743271223
F1 Score: 0.11239399869536856

weighted
Precision: 0.7579661016949153
Recall: 0.28
F1 Score: 0.19208414872798435

Mean FNR: 0.8616977225672877
Mean TNR: 0.9417711346833816
Mean FPR: 0.058228865316618444
Mean TPR: 0.13830227743271223
